### 数据格式

    DROP TABLE IF EXISTS dt_golf CASCADE;
    CREATE TABLE dt_golf (
        id integer NOT NULL,
        outside text,
        temperature double precision,
        humidity double precision,
        windy boolean,
        class text
    );
    INSERT INTO dt_golf VALUES (1,'sunny', 85, 85,'false','Don''t Play'),
    (2, 'sunny', 80, 90,'true', 'Don''t Play'),
    (3, 'overcast', 83, 78, 'false', 'Play'),
    (4, 'rain', 70, 96, 'false', 'Play'),
    (5, 'rain', 68, 80, 'false', 'Play'),
    (6, 'rain', 65, 70, 'true', 'Don''t Play'),
    (7, 'overcast', 64, 65, 'true', 'Play'),
    (8, 'sunny', 72, 95, 'false', 'Don''t Play'),
    (9, 'sunny', 69, 70, 'false', 'Play'),
    (10, 'rain', 75, 80, 'false', 'Play'),
    (11, 'sunny', 75, 70, 'true', 'Play'),
    (12, 'overcast', 72, 90, 'true', 'Play'),
    (13, 'overcast', 81, 75,'false', 'Play'),
    (14, 'rain', 71, 80, 'true', 'Don''t Play');

### 训练数据
http://madlib.apache.org/docs/latest/group__grp__decision__tree.html#train

    select madlib.tree_train(
        'dt_golf',  --输入训练集合
        'dt_golf_model',  --输出模型表名
        'id',       --表必须有id,为了交叉验证
        'class', --因变量
        'outside,temperature,humidity,windy',--自变量
        null::text,
        'gini', --分割准则,默认基尼系数,产生最小 "基尼系数" 的特征被选为 分割点 进行分割
        null,
        null,
        5,  --最大树的深度,不要超过"自变量"数量
        3, --最小分割数,节点中样本小于这个数不会 被分割,min_split
        1, --"叶子节点层" 最小的样本数,一般是min_bucket/3
        10,--连续 变量 离散化 的分割区间,
        'cp=0,n_folds=6'  --n_folds交叉验证的折数,cp??
        );

## 决策数的显示 
    select madlib.tree_display('dt_golf_model',true); --输入为dot文件,读取后保存为文件(my.dot)
    dot -Tjpg my.dot -o my.jpg
 <img src='img/my.jpg' />

## 使用决策数预测

### 1.数据表 结构同训练数据集一样
    DROP TABLE IF EXISTS dt_golf_new CASCADE;
    CREATE TABLE dt_golf_new (
        id integer NOT NULL,
        outside text,
        temperature double precision,
        humidity double precision,
        windy boolean
    );
    INSERT INTO dt_golf_new VALUES (1,'sunny', 32, 85,'false'),
    (2, 'sunny', 80, 90,'true'),
    (3, 'overcast', 83, 78, 'true');

### 2.预测结果

    select madlib.tree_predict('dt_golf_model',  --模型表
                  'dt_golf_new', --要预测的数据
                 'dt_golf_new_result', --结果表 
                 'response'); --输入的type,response表示分类结果,prob表示每个类别的概率
                 
    select * from dt_golf_new_result;--查看结果
<img src='img/rf1.png'>

## 随机森林

### 1.数据格式与随机树一直

### 2.训练

    select madlib.forest_train('dt_golf',  --输入训练集合
                 'dt_golf_forest_model',  --输出模型表名
                 'id',       --表必须有id,为了交叉验证
                 'class', --因变量
                 'outside,temperature,humidity,windy',--自变量
                 null::text, --排除的变量,例如排除 windy,humidity
                 null,
                 10, --num_trees
                 2,--随机选择的特征,不同于决策树,为了防止过拟合,每一次分裂的特征随机选取,一般sqrt(num features)
                 true::boolean,--是否计算特征  重要性
                 1,--1
                 8,--最大树的深度,不要超过"自变量"数量
                 3, --最小分割数,节点中样本小于这个数不会 被分割,min_split
                 1, --"叶子节点层" 最小的样本数,一般是min_split/3
                 10 --连续 变量 离散化 的分割区间,
                 );
     模型参数表dt_golf_forest_model有gid,sample_id,tree组成,分别代表分组id,森林中的决策树id,生成的树

### 3.随机森林显示
    select madlib.get_tree( 'dt_golf_forest_model',1,2,true); --输出第二棵树,dot格式

### 2.预测结果

    select madlib.forest_predict('dt_golf_forest_model',  --模型表
                  'dt_golf_new', --要预测的数据
                 'dt_golf_forest_new_result', --结果表 
                 'response'); --输入的type,response表示分类结果,prob表示每个类别的概率
                 
    select * from dt_golf_forest_new_result;--查看结果
<img src='img/rf1.png'>

## KNN(K临近值)

### 1.训练数据集:
    1.类别的字段需要one-hot编码,这里省略了...
    2.数值字段要缩放到[0,1],这里省略了...

    DROP TABLE IF EXISTS dt_golf_train CASCADE;
    CREATE TABLE dt_golf_train(
        id integer NOT NULL,
        data float8[],  ---不同于决策树,这里要使用数组
        class int
    );
    
    INSERT INTO dt_golf_train VALUES (1,'{0,85,85,0}',0),
    (2, '{0,80,90,1}', 0),
    (3, '{1,83,78,0}', 1),
    (4, '{2,70,96,0}', 1),
    (5, '{2,68,80,0}', 1),
    (6, '{2,65,70,1}', 0),
    (7, '{1,64,65,1}', 1),
    (8, '{0,72,95,0}', 0),
    (9,  '{0,69,70,0}',1),
    (10, '{2,75,80,0}',1);

    DROP TABLE IF EXISTS dt_golf_test CASCADE;
    CREATE TABLE dt_golf_test(
        id integer NOT NULL,
        data float8[],  ---不同于决策树,这里要使用数组
        class text
    );
    INSERT INTO dt_golf_test VALUES (1, '{0,75,70,1}',1),
    (2, '{1,72,90,1}', 1),
    (3, '{1,72,90,1}', 1),
    (4, '{2,71,80,1}', 0);

### 预测数据

    select madlib.knn(
        'dt_golf_train',--源表
        'data', --特征变量
        'id',
        'class',--自变量
        'dt_golf_test',--测试表
        'data', --特征变量
        'id',
        'dt_golf_knn',--输出表
        3 --K值
    );
    select * from dt_golf_test;